In [7]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
paper_dir             = config.get_property('paper_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")


COLOR_LIST1 = ["#F8AFA8", "#FDDDA0", "#F5CDB4", "#74A089"]

In [8]:
adht_ward_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "counts_ward.csv" ), parse_dates=['date'])

date_start = pd.to_datetime('2020-02-01')
date_end   = pd.to_datetime('2021-02-28')
dates_sim  = pd.date_range(date_start, date_end)

adht_ward_df   = adht_ward_df[adht_ward_df.date.isin(dates_sim)]
#selected_ward = ['Allen Hospital', 'Harkness Pavilion', 'Milstein Hospital', 'Mschony', 'Presbyterian Hospital']

A_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_admitted')
D_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_discharged')
H_df     = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_hospitalized')
tests_df = pd.pivot(adht_ward_df, index='ward', columns='date', values='num_tested')

pop        = H_df.mean(axis=1)
num_pop    = len(pop)
ward_names = pop.index

ward_num          = len(ward_names)
ward_transfers_df = pd.read_csv(os.path.join(data_db_dir, "long_files_8_25_2021", "transfers_ward.csv"), parse_dates=['date'])
ward_transfers_df = ward_transfers_df[ward_transfers_df.date.isin(dates_sim)]

M_df = np.zeros((ward_num, ward_num, len(dates_sim)+1))

for i in range(ward_num):
    ward_from = ward_names[i]
    for j in range(ward_num):
        ward_to      = ward_names[j]
        transfers_ij = ward_transfers_df[(ward_transfers_df.ward_from==ward_from) & (ward_transfers_df.ward_to==ward_to)]

        if(transfers_ij.shape[0] > 0) :
            dates_ij                = transfers_ij.date.values
            dates_ind               = np.where(np.in1d(dates_ij, dates_sim))[0]
            transfered              = transfers_ij.num_transfered.values
            M_df[i, j, dates_ind-1] = transfered

In [13]:
from models import process_metapop, observe_metapop, init_metapop

if_settings = {
   "Nif"                : 50,          # number of iterations of the IF
   "type_cooling"       : "geometric", # type of cooling schedule
   "shrinkage_factor"   : 0.9,         # shrinkage factor for the cooling schedule
   "inflation"          : 1.01,        # inflation factor for spreading the variance after the EAKF step
}

model_settings = {
    "param_name"  : ["ρ", "β"],   # importation and transmission rate
    "p"           : 2,              # number of parameters
    "k"           : num_pop,        # number of observations | We are just observing carriage
    "n"           : 3*num_pop,      # number of state variables / dimension of the state space
    "dt"          : 1,              # time step
    "T"           : len(dates_sim), # time to run
    "m"           : 1000,           # number of ensembles
    "stochastic"  : True,           # is stochastic
    "num_pop"     : num_pop,
    "dates"       : dates_sim
    }

p = model_settings["p"]
m = model_settings["m"]
T = model_settings["T"]

gamma = 0.20
delta = 1/120  # decolonization rate

A = A_df.to_numpy()
D = D_df.to_numpy()
H = H_df.to_numpy()
M = M_df

#tests = tests_df.to_numpy()
tests = np.zeros((num_pop,T))
tests = tests_df

# used mean values
#A = np.array([np.repeat(np.mean(A[i,:]),A.shape[1]) for i in np.arange(num_pop)])
#D = np.array([np.repeat(np.mean(D[i,:]),D.shape[1]) for i in np.arange(num_pop)])
#H = np.array([np.repeat(np.mean(H[i,:]),H.shape[1]) for i in np.arange(num_pop)])

# Process model for the ifeakf | model(x, gamma, beta, delta, rho, sigma, pop, m=1, stochastic=True)
process_model = lambda t, x, θ, model_settings : process_model(t, x,
                                        gamma          = [gamma]*model_settings["m"],
                                        beta           = θ[1, :],
                                        delta          = delta,
                                        Nmean          = pop,
                                        N              = H[:,  t],
                                        A              = A[:,  t],
                                        D              = D[:,  t],
                                        M              = M[:,:,t],
                                        model_settings = model_settings)

# Observational model for the ifeakf |  g(t, x, rho)
observational_model  = lambda t, x, θ, model_settings: observe_metapop(
                                        t,
                                        x,
                                        N=H[:,t],
                                        rho=θ[0, :],
                                        tests=tests[:,t],
                                        model_settings=model_settings)

# f0 model for the ifeakf            | initial_condition(c0, pop=2000, m=300)
initial_guess_x0     = lambda θ, model_settings:  init_metapop(
                                        N0 = H[:,0],                      # population size
                                        c0 = [gamma]*model_settings["m"], # importation rate
                                        model_settings=model_settings)


ρ_min = 0.01  # test sensitivity minimum
ρ_max = 0.5  # test sensitivity maximum

βmin = 0.00  # transmission rate minimum
βmax = 0.5   # transmission rate maximum

max_total_pop     = np.max(H.sum(axis=0))
state_space_range = np.array([0, max_total_pop])
parameters_range  = np.array([[ρ_min, ρ_max],
                              [βmin, βmax]])

σ_perturb         = np.array([(ρ_max-ρ_min)/4, (βmax-βmin)/4])


2777